In [1]:
import pandas as pd
import numpy as np

In [12]:
data = pd.read_csv('dataset_tweet_sentiment_pilkada_DKI_2017.csv', delimiter = ';')
data.columns = ["label", "tweet"]
data.head()

,label,tweet
0,1,Banyak akun kloning seolah2 pendukung #agussil...
1,2,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,4,Kasian oh kasian dengan peluru 1milyar untuk t...
4,5,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


In [13]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data['tweet'] = data['tweet'].str.lower()


print('Case Folding Result : \n')
print(data['tweet'].head(5))
print('\n\n\n')

Case Folding Result : 

0    banyak akun kloning seolah2 pendukung #agussil...
1    #agussilvy bicara apa kasihan yaa...lap itu ai...
2    kalau aku sih gak nunggu hasil akhir qc tp lag...
3    kasian oh kasian dengan peluru 1milyar untuk t...
4    maaf ya pendukung #agussilvy..hayo dukung #ani...
Name: tweet, dtype: object






In [14]:
# Tokenizing

import string 
import re #regex library
import nltk
nltk.download('punkt')

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data['tweet'] = data['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['tweet'] = data['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['tweet'] = data['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['tweet'] = data['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['tweet'] = data['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['tweet'] = data['tweet'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['tweet_tokens'] = data['tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [banyak, akun, kloning, seolah, pendukung, mul...
1    [bicara, apa, kasihan, yaalap, itu, air, matan...
2    [kalau, aku, sih, gak, nunggu, hasil, akhir, q...
3    [kasian, oh, kasian, dengan, peluru, milyar, u...
4       [maaf, ya, pendukung, hayo, dukung, diputaran]
Name: tweet_tokens, dtype: object






[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADVENTA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
data.head()

,label,tweet,tweet_tokens
0,1,banyak akun kloning seolah pendukung mulai men...,"[banyak, akun, kloning, seolah, pendukung, mul..."
1,2,bicara apa kasihan yaalap itu air matanya wkwk...,"[bicara, apa, kasihan, yaalap, itu, air, matan..."
2,3,kalau aku sih gak nunggu hasil akhir qc tp lag...,"[kalau, aku, sih, gak, nunggu, hasil, akhir, q..."
3,4,kasian oh kasian dengan peluru milyar untuk ti...,"[kasian, oh, kasian, dengan, peluru, milyar, u..."
4,5,maaf ya pendukung hayo dukung diputaran,"[maaf, ya, pendukung, hayo, dukung, diputaran]"


In [16]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

data['tweet_tokens_fdist'] = data['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(data['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(banyak, 1), (akun, 1), (kloning, 1), (seolah...
1    [(bicara, 1), (apa, 1), (kasihan, 1), (yaalap,...
2    [(nunggu, 2), (kalau, 1), (aku, 1), (sih, 1), ...
3    [(kasian, 2), (oh, 1), (dengan, 1), (peluru, 1...
4    [(maaf, 1), (ya, 1), (pendukung, 1), (hayo, 1)...
Name: tweet_tokens_fdist, dtype: object


In [17]:
# Filtering (Stopword Removal)

from nltk.corpus import stopwords
nltk.download('stopwords')

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['tweet_tokens_WSW'] = data['tweet_tokens'].apply(stopwords_removal) 


print(data['tweet_tokens_WSW'].head())

0    [akun, kloning, pendukung, menyerang, paslon, ...
1    [bicara, kasihan, yaalap, air, matanya, wkwkwkwk]
2    [nunggu, hasil, qc, tp, nunggu, motif, cuitan,...
3    [kasian, oh, kasian, peluru, milyar, rw, mempa...
4           [maaf, pendukung, hayo, dukung, diputaran]
Name: tweet_tokens_WSW, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADVENTA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Normalization

normalizad_word_dict = {}

for index, row in data.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data['tweet_normalized'] = data['tweet_tokens_WSW'].apply(normalized_term)

data['tweet_normalized'].head(10)

0    [akun, kloning, pendukung, menyerang, paslon, ...
1    [bicara, kasihan, yaalap, air, matanya, wkwkwkwk]
2    [nunggu, hasil, qc, tp, nunggu, motif, cuitan,...
3    [kasian, oh, kasian, peluru, milyar, rw, mempa...
4           [maaf, pendukung, hayo, dukung, diputaran]
5    [aneh, deh, lebay, sangkut, pautkan, kandidat,...
6    [allah, swt, ucapkan, syukur, terimakasih, mak...
7    [terima, kasih, teruntuk, pendukung, kalianlah...
8    [trima, kasih, keistiqomahan, relawan, tlh, me...
9                       [mengenang, pidato, kekalahan]
Name: tweet_normalized, dtype: object

In [27]:
!pip install swifter
!pip install sastrawi
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['tweet_tokens_stemmed'] = data['tweet_normalized'].swifter.apply(get_stemmed_term)
print(data['tweet_tokens_stemmed'])

2672
------------------------
akun : akun
kloning : kloning
pendukung : dukung
menyerang : serang
paslon : paslon
opini : opini
argumen : argumen
pmbenaranjangan : pmbenaranjangan
terkecoh : kecoh
bicara : bicara
kasihan : kasihan
yaalap : yaalap
air : air
matanya : mata
wkwkwkwk : wkwkwkwk
nunggu : nunggu
hasil : hasil
qc : qc
tp : tp
motif : motif
cuitan : cuit
kayak : kayak
pasca : pasca
nyungsep : nyungsep
kasian : kasi
oh : oh
peluru : peluru
milyar : milyar
rw : rw
mempan : mempan
menangin : menangin
pilkada : pilkada
maaf : maaf
hayo : hayo
dukung : dukung
diputaran : putar
aneh : aneh
deh : deh
lebay : lebay
sangkut : sangkut
pautkan : paut
kandidat : kandidat
calgub : calgub
allah : allah
swt : swt
ucapkan : ucap
syukur : syukur
terimakasih : terimakasih
makna : makna
suara : suara
terima : terima
kasih : kasih
teruntuk : untuk
kalianlah : kalian
pembeda : beda
kali : kali
trima : trima
keistiqomahan : keistiqomahan
relawan : rawan
tlh : tlh
mengantarkan : antar
signifikan : s

rachland : rachland
nashidik : nashidik
menyadari : sadar
musuh : musuh
sarung : sarung
selfienya : selfienya
motivasi : motivasi
terselubung : selubung
hmmm : hmmm
amieeennnnn : amieeennnnn
lupa : lupa
janji : janji
manis : manis
kampanye : kampanye
sejati : sejati
target : target
fitnah : fitnah
akunakun : akunakun
mentah : mentah
dikarbit : dikarbit
demokrat : demokrat
kemenangan : menang
pemilu : milu
memakai : pakai
sentimen : sentimen
sara : sara
akibat : akibat
melorot : lorot
enam : enam
kubu : kubu
serang : serang
dicontohkan : contoh
pendukungnya : dukung
paska : paska
putaran : putar
jakarta : jakarta
perentangan : rentang
tajam : tajam
berjalan : jalan
menjauh : jauh
hiruk : hiruk
pikuk : pikuk
melanjutkan : lanjut
perjalanan : jalan
kecewa : kecewa
berpidato : pidato
timnya : tim
terdiam : diam
menangis : menang
melepaskan : lepas
sejujurnya : jujur
penilaian : nilai
subyektif : subyektif
cepat : cepat
catatan : catat
negatif : negatif
tolak : tolak
bajingan : bajing
tengi

goal : goal
improvisasi : improvisasi
ahydi : ahydi
yra : yra
klop : klop
dah : dah
diperhitungkan : hitung
diharapkan : harap
ri : ri
membangun : bangun
tolong : tolong
hargai : harga
pilkades : pilkades
rame : rame
kaos : kaos
dibagibagikan : dibagibagikan
dilingkungan : lingkung
utan : utan
kayu : kayu
jaktim : jaktim
ahysylvi : ahysylvi
terlaksana : laksana
tentukan : tentu
pro : pro
mudah : mudah
silvy : silvy
mio : mio
helm : helm
komplek : komplek
pemilihan : pilih
dijamin : jamin
segan : segan
bersatu : satu
menangkan : menang
jiwa : jiwa
sandiapa : sandiapa
ucapan : ucap
peran : peran
loyalis : loyalis
cakeeepppp : cakeeepppp
sinyal : sinyal
merapat : rapat
rapat : rapat
mengakui : aku
apapun : apa
tujuannya : tuju
great : great
applause : applause
patut : patut
ditiru : tiru
demokrasi : demokrasi
salut : salut
mpok : mpok
quick : quick
countttp : countttp
kpud : kpud
youtubebxrzdua : youtubebxrzdua
tenang : tenang
bening : bening
tulus : tulus
masukan : masuk
kritik : kritik


abused : abused
islamist : islamist
radicals : radicals
deddy : deddy
corbuzier : corbuzier
bawa : bawa
ciduk : ciduk
takjub : takjub
geli : geli
kalimatkalimat : kalimatkalimat
khilaf : khilaf
disayang : sayang
ayang : ayang
diajak : ajak
komitment : komitment
ujungujungnya : ujungujungnya
nyesel : nyesel
kompak : kompak
memukul : pukul
nuntut : nuntut
belom : bom
maunya : mau
enak : enak
rapi : rapi
bukti : bukti
begtiu : begtiu
dicintai : cinta
warganyanamun : warganyanamun
terhornat : terhornat
kelicikan : kelici
menjijikan : jijik
lawanya : lawa
stress : stress
dipikiran : pikir
duit : duit
itung : itung
rp : rp
dananya : dana
melebihi : lebih
materi : materi
sarankan : saran
pansus : pansus
undang : undang
lsm : lsm
pencari : cari
fakta : fakta
gila : gila
dirusak : rusak
komen : komen
settingan : settingan
pencitraan : citra
disuruh : suruh
hahaha : hahaha
kaga : kaga
bahas : bahas
yaaa : yaaa
perhatian : perhati
stres : stres
kembang : kembang
dibohongi : bohong
menanglah : men

bloon : bloon
harum : harum
ungkapan : ungkap
bahasa : bahasa
abal : abal
kantor : kantor
kehabisan : habis
medan : medan
indah : indah
datangkan : datang
bungapak : bungapak
diakhir : akhir
jabatan : jabat
sadar : sadar
berkati : berkat
mengalir : alir
wisata : wisata
riang : riang
penuhnya : penuh
subhanallah : subhanallah
unik : unik
viral : viral
medsos : medsos
satunya : satu
memberkati : kati
marah : marah
galakgalak : galakgalak
nongkrong : nongkrong
masang : masang
jogja : jogja
papan : papan
jahat : jahat
ninggalin : ninggalin
terlanjur : lanjur
antrian : antri
il : il
phenomenon : phenomenon
seorg : seorg
figur : figur
batam : batam
gusur : gusur
jutaan : juta
pendidikan : didik
totorqq : totorqq
pusing : pusing
tenangkonsentrasiberdoaselalu : tenangkonsentrasiberdoaselalu
bahagia : bahagia
doanya : doa
pa : pa
kyai : kyai
miliki : milik
arrow : arrow
trophy : trophy
oke : oke
peach : peach
tohh : tohh
disengaja : sengaja
sepaket : paket
risau : risau
hahahahahaha : hahahahah

aher : aher
hhmm : hhmm
dilanda : landa
panitianya : panitia
panitia : panitia
main : main
licik : licik
diserang : serang
habishabisan : habishabisan
ahox : ahox
kl : kl
nyalon : nyalon
kab : kab
kyknya : kyknya
entrepreneurship : entrepreneurship
education : education
kekuatan : kuat
mei : mei
perhatikan : perhati
aniessandikaos : aniessandikaos
provokatif : provokatif
rekonsiliasi : rekonsiliasi
visi : visi
misi : misi
musolah : musolah
pilkadatiap : pilkadatiap
pengajian : aji
angsuran : angsur
th : th
sengsarain : sengsarain
kabur : kabur
babak : babak
belur : bur
konfirmasinya : konfirmasi
ngeles : ngeles
binimuda : binimuda
kw : kw
aspal : aspal
urus : urus
istri : istri
bor : bor
sumur : sumur
ngebor : ngebor
lagimulai : lagimulai
kaku : kaku
tangannya : tangan
nginulpun : nginulpun
without : without
manies : manies
bertabur : tabur
wacana : wacana
sandiwara : sandiwara
mah : mah
tajir : tajir
melintir : melintir
alhamdullilah : alhamdullilah
intelijen : intelijen
menanggapinya

Pandas Apply:   0%|          | 0/900 [00:00<?, ?it/s]

0      [akun, kloning, dukung, serang, paslon, opini,...
1         [bicara, kasihan, yaalap, air, mata, wkwkwkwk]
2      [nunggu, hasil, qc, tp, nunggu, motif, cuit, k...
3      [kasi, oh, kasi, peluru, milyar, rw, mempan, m...
4                    [maaf, dukung, hayo, dukung, putar]
                             ...                        
895          [kali, bpk, lihat, rspun, selfie, ok, hand]
896    [rangkul, orang, batas, usia, kelamin, ok, han...
897    [jago, bidang, digital, smiling, face, with, s...
898                                                   []
899    [sandiaga, bangun, rumah, dp, simpel, banding,...
Name: tweet_tokens_stemmed, Length: 900, dtype: object


In [28]:
data.to_csv("Pemiluu.csv")